In [21]:
import requests
import json
import pandas as pd
from datetime import datetime
api_key = 'f7895bc3ff4930480444c1033ddbe37f'

SPORT = 'basketball_ncaab' # use the sport_key from the /sports endpoint below, or use 'upcoming' to see the next 8 games across all sports

HOST = f'https://api.the-odds-api.com'

REGIONS = 'us' # uk | us | eu | au. Multiple can be specified if comma delimited

MARKETS = 'h2h, spreads, totals, outrights' # h2h | spreads | totals. Multiple can be specified if comma delimited

ODDS_FORMAT = 'american' # decimal | american

DATE_FORMAT = 'iso'

sports_response = requests.get(
    f'{HOST}/v4/sports',
    params={
        'api_key': api_key,
        'regions': REGIONS,
        'markets': MARKETS,
        'oddsFormat': ODDS_FORMAT,
        'dateFormat': DATE_FORMAT,
    }
)
odds_response = requests.get(
    f'{HOST}/v4/sports/{SPORT}/odds',
    params={
        'api_key': api_key,
        'regions': REGIONS,
        'markets': MARKETS,
        'oddsFormat': ODDS_FORMAT,
        'dateFormat': DATE_FORMAT,
    }
)

if odds_response.status_code != 200:
    print(f'Failed to get odds: status_code {odds_response.status_code}, response body {odds_response.text}')

else:
    odds_json = odds_response.json()

Failed to get odds: status_code 422, response body {"message":"One or more markets invalid. Check the docs https://the-odds-api.com/liveapi/guides/v4/"}



In [36]:
sports_keys = pd.json_normalize(sports_response.json())['key'].to_list()

odds_responses = {
    sport: requests.get(
    f'{HOST}/v4/sports/{sport}/odds',
    params={
        'api_key': api_key,
        'regions': REGIONS,
        'oddsFormat': ODDS_FORMAT,
        'dateFormat': DATE_FORMAT,
    } 
).json() for sport in sports_keys
}


In [37]:
odds_responses

{'americanfootball_ncaaf': [{'id': '329ee4daba3514dce8018d5768e5043f',
   'sport_key': 'americanfootball_ncaaf',
   'sport_title': 'NCAAF',
   'commence_time': '2024-08-24T18:30:00Z',
   'home_team': 'Georgia Tech Yellow Jackets',
   'away_team': 'Florida State Seminoles',
   'bookmakers': [{'key': 'fanduel',
     'title': 'FanDuel',
     'last_update': '2024-02-17T09:14:16Z',
     'markets': [{'key': 'h2h',
       'last_update': '2024-02-17T09:14:16Z',
       'outcomes': [{'name': 'Florida State Seminoles', 'price': -480},
        {'name': 'Georgia Tech Yellow Jackets', 'price': 360}]}]}]},
  {'id': '6d4fa722a24acf375db350092cf688b9',
   'sport_key': 'americanfootball_ncaaf',
   'sport_title': 'NCAAF',
   'commence_time': '2024-08-31T19:30:00Z',
   'home_team': 'Colorado Buffaloes',
   'away_team': 'North Dakota State Bison',
   'bookmakers': [{'key': 'fanduel',
     'title': 'FanDuel',
     'last_update': '2024-02-17T09:14:16Z',
     'markets': [{'key': 'h2h',
       'last_update': '

In [30]:
SPORT = "americanfootball_ncaaf_championship_winner"
ncaaf_champ_odds_response = requests.get(
    f'{HOST}/v4/sports/{SPORT}/odds',
    params={
        'api_key': api_key,
        'regions': REGIONS,
        'oddsFormat': ODDS_FORMAT,
        'dateFormat': DATE_FORMAT,
    }
)

ncaaf_champ_odds_response.json()

[{'id': 'be95dd8b58f4a0c6de52611048d644bc',
  'has_outrights': True,
  'sport_key': 'americanfootball_ncaaf_championship_winner',
  'sport_title': 'NCAAF Championship Winner',
  'commence_time': '2025-01-15T01:00:00Z',
  'home_team': None,
  'away_team': None,
  'bookmakers': [{'key': 'betonlineag',
    'title': 'BetOnline.ag',
    'last_update': '2024-02-17T09:09:45Z',
    'markets': [{'key': 'outrights',
      'last_update': '2024-02-17T09:09:45Z',
      'outcomes': [{'name': 'Georgia Bulldogs', 'price': 350},
       {'name': 'Ohio State Buckeyes', 'price': 550},
       {'name': 'Texas Longhorns', 'price': 700},
       {'name': 'Oregon Ducks', 'price': 800},
       {'name': 'Alabama Crimson Tide', 'price': 1100},
       {'name': 'Ole Miss Rebels', 'price': 1400},
       {'name': 'Michigan Wolverines', 'price': 1600},
       {'name': 'LSU Tigers', 'price': 1600},
       {'name': 'Florida State Seminoles', 'price': 2200},
       {'name': 'Penn State Nittany Lions', 'price': 3300},
    

In [13]:


odds_pd = pd.json_normalize(
    odds_json, 
    meta = [
    'id', 
    'sport_key', 
    'sport_title', 
    'commence_time', 
    'home_team', 
    'away_team',
    ['key', 'title']
    ],
    meta_prefix= 'top_level.',
    record_path = ['bookmakers', 'markets', 'outcomes']
    )

odds_pd = pd.merge(
    odds_pd,
    odds_pd.groupby(
    ['top_level.id', 'top_level.key.title']
    )['price'].max().reset_index().rename(columns = {'price': 'max_price'}),
    how = 'left',
    on = ['top_level.id', 'top_level.key.title']
)

odds_pd = pd.merge(
    odds_pd,
    odds_pd.groupby(
    ['top_level.id', 'top_level.key.title']
    )['price'].min().reset_index().rename(columns = {'price': 'min_price'}),
    how = 'left',
    on = ['top_level.id', 'top_level.key.title']
)

odds_pd.loc[odds_pd['price'] == odds_pd['min_price'], 'team_type'] = 'favorite'
odds_pd.loc[odds_pd['price'] == odds_pd['max_price'], 'team_type'] = 'underdog'


odds_pd['odds_id'] = [value1 + value2 for (value1, value2) in zip(
    odds_pd['top_level.id'].astype('str').to_list(), 
    odds_pd['top_level.key.title'].astype('str').to_list()
    )]

wide_pd = pd.pivot(odds_pd.reset_index(), 
    index = ['odds_id', 'name'],
    columns = ['team_type'],
    values = ['price']
).reset_index()
wide_pd.columns = [''.join(col) for col in wide_pd.columns.values]

wide_pd = wide_pd.groupby('odds_id').agg({'pricefavorite': 'max', 'priceunderdog': 'max'}).reset_index()

wide_pd['favorite_team_name'] = pd.merge(
    wide_pd,
    odds_pd[odds_pd['team_type'] == 'favorite'],
    how = 'left',
    on = 'odds_id'
)['name']

wide_pd['underdog_team_name'] = pd.merge(
    wide_pd,
    odds_pd[odds_pd['team_type'] == 'underdog'],
    how = 'left',
    on = 'odds_id'
)['name']

wide_pd = pd.merge(
    wide_pd,
    odds_pd.groupby('odds_id').agg({
    'top_level.sport_key': 'max',
    'top_level.sport_title': 'max',
    'top_level.commence_time': 'max',
    'top_level.away_team': 'max',
    'top_level.home_team': 'max',
    'top_level.key.title': 'max',
    'top_level.id': 'max'
}).reset_index(),
how = 'left',
on = 'odds_id')

wide_pd = wide_pd.rename(columns = {
    'pricefavorite': 'price_favorite',
    'priceunderdog': 'price_underdog',
    'top_level.sport_key': 'sport_key',
    'top_level.sport_title': 'sport_title',
    'top_level.commence_time': 'game_start_time',
    'top_level.away_team': 'away_team',
    'top_level.home_team': 'home_team',
    'top_level.key.title': 'bookie',
    'top_level.id': 'game_id'
    })

wide_pd.loc[pd.to_datetime(wide_pd['game_start_time']).dt.tz_localize(None) < datetime.utcnow(), 'live_ind'] = True
wide_pd.loc[pd.to_datetime(wide_pd['game_start_time']).dt.tz_localize(None) >= datetime.utcnow(), 'live_ind'] = False

# filter out live games
wide_pd = wide_pd[wide_pd['live_ind'] == False]


best_spread_pd = wide_pd.groupby(['game_id', 'home_team', 'away_team', 'sport_title', 'game_start_time', 'sport_key']).agg({
    'price_favorite': 'max', 
    'price_underdog': 'max'
    }).reset_index()

best_spread_pd = pd.merge(best_spread_pd,
                          wide_pd.groupby(['game_id', 'price_favorite']).agg({'bookie': lambda x: ', '.join(x)}).reset_index().rename(columns = {'bookie': 'price_favorite_bookies'}),
                          how = 'left', 
                          on = ['game_id', 'price_favorite'])

best_spread_pd = pd.merge(best_spread_pd,
                          wide_pd.groupby(['game_id', 'price_underdog']).agg({
    'bookie': lambda x: ', '.join(x)
    }).reset_index().rename(columns = {'bookie': 'price_underdog_bookies'}),
                          how = 'left', 
                          on = ['game_id', 'price_underdog'])

best_spread_pd['favorite_implied_prob'] = (abs(best_spread_pd['price_favorite']))/(abs(best_spread_pd['price_favorite'])+100)
best_spread_pd['underdog_implied_prob'] = 100/(abs(best_spread_pd['price_underdog'])+100)

best_spread_pd['arb_perc'] = best_spread_pd['favorite_implied_prob'] + best_spread_pd['underdog_implied_prob']

best_spread_pd['ud_over_fav_bet_proportion'] = best_spread_pd['underdog_implied_prob']/best_spread_pd['favorite_implied_prob']

In [14]:
best_spread_pd

,game_id,home_team,away_team,sport_title,game_start_time,sport_key,price_favorite,price_underdog,price_favorite_bookies,price_underdog_bookies,favorite_implied_prob,underdog_implied_prob,arb_perc,ud_over_fav_bet_proportion
0,078070dc05d0e2ec50775183819cfd72,North Carolina Tar Heels,Virginia Tech Hokies,NCAAB,2024-02-17T19:00:00Z,basketball_ncaab,-720.0,500.0,FanDuel,"FanDuel, PointsBet (US)",0.878049,0.166667,1.044715,0.189815
1,084c2c5e48cf85a7b81ebd3a29b377be,Duquesne Dukes,Saint Joseph's Hawks,NCAAB,2024-02-17T19:30:00Z,basketball_ncaab,-180.0,152.0,PointsBet (US),FanDuel,0.642857,0.396825,1.039683,0.617284
2,0a725739511f35563b0d4084206f0019,Detroit Mercy Titans,Fort Wayne Mastodons,NCAAB,2024-02-17T18:00:00Z,basketball_ncaab,-465.0,375.0,FanDuel,PointsBet (US),0.823009,0.210526,1.033535,0.255801
3,0cf434987ba19293a2a5f8377c169951,La Salle Explorers,Massachusetts Minutemen,NCAAB,2024-02-17T19:00:00Z,basketball_ncaab,-210.0,175.0,"FanDuel, PointsBet (US)",PointsBet (US),0.677419,0.363636,1.041056,0.536797
4,16a389474eb307f58f24ad4f4642f27a,Dayton Flyers,Fordham Rams,NCAAB,2024-02-17T18:30:00Z,basketball_ncaab,-1799.0,1050.0,PointsBet (US),PointsBet (US),0.947341,0.086957,1.034297,0.091790
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,e8a10f91d236f6b1861d7e49fc24b038,Columbia Lions,Dartmouth Big Green,NCAAB,2024-02-17T00:00:00Z,basketball_ncaab,-455.0,440.0,Caesars,"BetOnline.ag, LowVig.ag",0.819820,0.185185,1.005005,0.225885
60,ec3bbaeab9f2bb6126da752c82c1056e,Oklahoma St Cowboys,BYU Cougars,NCAAB,2024-02-17T19:00:00Z,basketball_ncaab,-275.0,225.0,"FanDuel, PointsBet (US)","BetRivers, Unibet",0.733333,0.307692,1.041026,0.419580
61,ef5756f5e73d1caa1425a973c0425b3d,Houston Cougars,Texas Longhorns,NCAAB,2024-02-17T18:00:00Z,basketball_ncaab,-800.0,550.0,"FanDuel, PointsBet (US)",PointsBet (US),0.888889,0.153846,1.042735,0.173077
62,f1040990e303311b27fac1ad1d743bfe,Ole Miss Rebels,Missouri Tigers,NCAAB,2024-02-18T01:30:00Z,basketball_ncaab,-625.0,490.0,"BetRivers, Unibet",FanDuel,0.862069,0.169492,1.031560,0.196610
